# Laboratorio di web scraping Saul Urso

### Note su funzionamento notebook
* Il notebook assume che sia presente nella propria directory corrente la directory "2013" contente il dataset. **Attenzione**: la directory non è nello zip fornito, data l'impossibilità di poter inviare un file di dimensioni così grandi su moodle.
* Riguardo l'esercizio **2.1**:
    * Ogni punto dell'esercizio ( a, b, ecc...) assume siano stati effettuati gli import e siano stati creati i dataframe dalle due celle sotto il titolo "Esercizio 2.1";
    * Si possono eseguire i vari punti in qualunque ordine (le celle all'interno di un punto vanno eseguite nell'ordine in cui sono presenti);
    * Per poter mostrare i risultati senza dover eseguire il notebook, alla fine di ogni punto è stata messa un'immagine del grafico prodotto.
* Riguardo l'esercizio **2.2**:
    * Per una corretta esecuzione eseguire le celle nell'ordine in cui sono presenti (**non serve** eseguire le celle antecedenti al titolo dell'esercizio);
    * **Attenzione**: l'esecuzione dell'algoritmo di clustering potrebbe richiedere del tempo (e/o richiede molta RAM), sul mio pc è durata 3 minuti ma su altri potrebbe essere molto peggio.

* Riguardo l'esercizio **2.3**:
    * L'esercizio assume che siano state ottenute le componenti connesse tramite il punto **2.2** e che sia stato generato il file "Components.json";
    * Per non dover eseguire il punto **2.2**, recuperare il file json a https://drive.google.com/file/d/1ZdLgBX-0f1so-CHWL3ubCVRe5AT1zaoI/view?usp=sharing e metterlo nella directory del noteboook.
    

## Attenzione

* eseguire la cella sottostante prima di procedere all'esecuzione delle restanti celle del notebook

In [ ]:
#questa cella rinomina le varie colonne del dataset, per poter essere utilizzate successivamente.
#Le tabelle con le colonne rinominate saranno salvate nella directory "dataset"
#che verrà creata se non è già presente.

import os
import pandas as pd

print('start')
df_inputs = pd.read_csv("2013/inputs.csv")
df_transactions = pd.read_csv("2013/transactions.csv")
df_outputs = pd.read_csv("2013/outputs.csv")
df_mapping = pd.read_csv("2013/mapAddr2Ids8708820.csv")

try:
    os.mkdir('dataset')
except FileExistsError:
    print('Directory già esiste')
    
df_inputs.columns= ['txId','prevTxId','prevTxpos']
df_outputs.columns= ['txId','position','addressId','amount','scripttype']
df_transactions.columns= ['timestamp','blockId','txId','isCoinbase','fee']
df_mapping.columns= ['hash','addressId']

df_inputs.to_csv("dataset/inputs.csv",index=False)
df_transactions.to_csv("dataset/transactions.csv",index=False)
df_outputs.to_csv("dataset/outputs.csv",index=False)
df_mapping.to_csv("dataset/mapAddr2Ids8708820.csv",index=False)

print('end')


# Esercizio 2.1

In [ ]:
#eseguire per 2.1

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import calendar

In [ ]:
#eseguire per 2.1

df_inputs = pd.read_csv("dataset/inputs.csv")
df_transactions = pd.read_csv("dataset/transactions.csv")
df_outputs = pd.read_csv("dataset/outputs.csv")
df_mapping = pd.read_csv("dataset/mapAddr2Ids8708820.csv")

In [ ]:
df_mapping.info()
df_mapping.head(5)

In [ ]:
df_inputs.info()
df_inputs.head(5)

In [ ]:
df_outputs.info()
df_outputs.head(5)

In [ ]:
df_transactions.info()
df_transactions.head(5)
df_transactions.tail(5)

## a) distribuzione del numero di transazioni per blocco (occupazione del blocco), nell’intero periodo temporale considerato

### Cosa fare
1. Prendere la tabella delle transazioni;
2. Raggruppare per numero blocco **blockId**;
3. Contare quante transazioni ci sono per blocco;
4. Plottare su istogramma dati (visto che plotto su istogramma basta avere il numero di transazioni per blocco, alle frequenze ci pensa lui)


In [ ]:
n=20 #ampiezza bin, rappresenta che dimensioni sono presenti in quel bin


block_counts= df_transactions.groupby("blockId").size().reset_index(name='counts') #2,3

#4

x= block_counts['counts']

plt.gcf().set_size_inches(25,10) #imposto dimesioni figura

#imposto vari titoli

plt.title("Distribuzione del numero di transazioni per blocco",fontsize=24)
plt.xlabel("Numero di transazioni",fontsize=18,)
plt.ylabel("Numero di blocchi",fontsize=18)

plt.hist(x,bins=round(max(x)/n),log=True,edgecolor='black') #plotto istogramma

plt.show()

#Le frequenze sono molto più sbilanciate in realtà, visto che sto utilizzando la scala logaritmica
#dalla documentazione-> log: If True, the histogram axis will be set to a log scale

![](ImmaginiLWS/2.1a.png)

## b) Evoluzione dell'occupazione dei blocchi nel tempo, considerando intervalli temporali di due mesi. In questo caso produrre un grafico che riporti il numero di transazioni medie per ogni periodo considerato

### Cosa fare

1. Prendere tabella transazioni
2. Raggruppare per numero blocco **blockId**;
3. Ottenere numero transazioni per blocco
4. Tenere conto di timestamp blocco ( tutte le transazioni quando raggruppo avranno stesso timestamp -> basta prenderne uno);
5. Creare lista contente medie sui vari intervalli (valori da plottare su asse y)
6. Plottare medie ottenute (lineplot)

In [ ]:
block_counts= df_transactions.groupby("blockId").size().reset_index(name='counts') #2,3
block_stamps = df_transactions.groupby("blockId")['timestamp'].first().reset_index(name='timestamp') #4

data = block_counts.merge(block_stamps,on= 'blockId') #probabilmente join non necessaria.
#la faccio per essere sicuro che timestamp e count combacino, fisto che ho fatto 2 query separate
# e l'ordinamento potrebbe essere diverso

In [ ]:
#5

#prendo il primo e l'ultimo timestamp
min_stamp=data['timestamp'].min()
max_stamp = data['timestamp'].max()

#li converto in date
start = datetime.datetime.utcfromtimestamp(min_stamp)
start= datetime.datetime(start.year,start.month,day=1) #metto la data di partenza a inizio del mese

end = datetime.datetime.utcfromtimestamp(max_stamp)
end= datetime.datetime(end.year+1,start.month,1) #allineo con inizio mese successivo, serve per pd.date_range


#creo lista contenente un mese ogni 2

#prima ottengo lista contenente tutti i mesi
dates_list= pd.date_range(start.date(),end.date(),freq='MS').to_pydatetime() 

#nel mentre preparo già parte dell'asse delle ascisse per il plot
x_ax= [el.date() for idx,el in enumerate(dates_list) if idx % 2 == 1]

#poi tolgo un mese alterno (intervalli di 2 mesi)
dates_list= [ el.date() for idx,el in enumerate(dates_list) if idx % 2 == 0] 

#converto colonna timestamp in date
data['dates']= pd.to_datetime(data['timestamp'],unit='s').dt.date


#creazione lista medie

mean_list = list()

for idx,el in enumerate(dates_list): #calcolo la media su 2 mesi dell'occupazione dei blocchi
   
    if el == end.date() : #ho finito
        break
    
    #prendo altro estremo dell'intervallo  
    next= dates_list[idx+1] 
    
    #creo maschera per selezione (date devono essere frai 2 estremi)
    mask= (data['dates']>= el) & (data['dates']<= next)
    my_query= data[mask] #seleziono tutti i blocchi compresi frai 2 mesi
    
    total_tra = my_query.counts.sum() #numero tot transazioni in intervallo
    
    block_num = my_query['blockId'].size #numero blocchi
    
    #aggiungo a lista media transazioni per blocco in quei 2 mesi
    mean_list.append(float(total_tra)/float(block_num)) 


In [ ]:
#6

#devo plottare medie su 2 mesi
#la media la riporto con ascissa a metà dell'intervallo dei 2 mesi (la data che divide l'intervallo a metà)
#per gli estremi (primo giorno e ultimo giorno) riporto la media di quel giorno

#inserisco i 2 estremi tra le ascisse
min_stamp=data['timestamp'].min()
start = datetime.datetime.utcfromtimestamp(min_stamp).date()
x_ax.insert(0,start)

max_stamp = data['timestamp'].max()
end = datetime.datetime.utcfromtimestamp(max_stamp).date()
x_ax.append(end)

#inserisco le 2 medie giornaliere (primo e ultimo giorno) fra le ordinate

my_query= data[data['dates']==start]
total_tra = my_query.counts.sum()
block_num = my_query['blockId'].size

mean_list.insert(0,float(total_tra)/float(block_num))

my_query= data[data['dates']==end]
total_tra = my_query.counts.sum()
block_num = my_query['blockId'].size

mean_list.append(float(total_tra)/float(block_num))

#effettuo plotting
plt.gcf().set_size_inches(25,10) #imposto dimesioni figura

plt.xticks(rotation=270) #mette date ruotate
plt.title('Evoluzione dell\'occupazione dei blocchi nel tempo',fontsize=24)
plt.xlabel('Mesi',fontsize=18)
plt.ylabel("Transazioni per blocco",fontsize=18)
plt.plot(x_ax,mean_list,marker='o')
plt.xticks(x_ax)
plt.show()

![](ImmaginiLWS/2.1b.png)

## c) Ammontare totale degli UTXO al momento dell’ultima transazione registrata nella blockchain considerata

Per essere UTXO bisogna che l'output di una transazione non sia input di un'altra.
Notiamo che ogni output mantiene in particolare 2 cose:
* il riferimento alla transazione che lo ha generato
* la sua posizione fra gli output di tale transazione

che lo identificano univocamente (nessun altro output può avere tale coppia uguale).
Lo stesso discorso vale per gli input.
Ma, un input non è altro che un output speso (non UTXO).
Quindi, per vedere se un output è UTXO, basta verificare che non ci sia un input con la stessa coppia (transazione di provenienza, posizione).

Ma allora basta effettuare una join right fra le due tabelle inputs e outputs (outputs a dx) su questa coppia, e le righe che avranno i campi di inputs a NaN saranno UTXO (l'output non compare mai come input).

### Cosa fare

1. Prendere le 2 tabelle inputs e outputs
2. Effettuare join right
3. Selezionare righe con attributi di input nulli
4. Effettuare somma su attributo **amount**

In [ ]:
#1,2
all_results= df_inputs.merge(df_outputs, how='right', left_on=['prevTxId','prevTxpos'], right_on= ['txId','position'])

#3, che attributo di df_inputs scelgo da controllare è indifferente, sono NaN nella stessa condizione
UTXO_table = all_results[all_results['prevTxId'].isnull()]

UTXO_table['amount'].sum() #4

## d) Distribuzione degli intervalli di tempo che intercorrono tra la transazione che genera un valore in output (UTXO) e quella che lo consuma, per gli output spesi nel periodo considerato.


### Cosa fare

Gli output spesi sono tutti quelli presenti nella tabella inputs, quindi:

1. Effettuare join inputs-transazioni su **prevTxId** (ottengo timestamp del blocco da cui proviene input);
2. Effettuare join inputs-transazioni su **txId** (ottengo timestamp blocco in cui vengo speso);
3. Fare la differenza, per ogni riga, dei 2 timestamp (ottengo intervallo di tempo);
4. Plottare su istogramma distribuzione della durata degli intervalli di tempo.

In [ ]:
#1

join1 = df_inputs.merge(df_transactions,how='inner', left_on= ['prevTxId'], right_on=['txId']) 
#join1 = join1[['txId_x','prevTxId','prevTxpos','timestamp']]
join1 = join1.rename({'timestamp' : 'prevTime', 'txId_x':'txId'},axis='columns')


In [ ]:
#2

join1 = join1.merge(df_transactions,how='inner', on= 'txId')
#join1 = join1[['txId','prevTxId','prevTxpos','prevTime','timestamp']]
join1.rename({'timestamp' : 'endTime'},axis = 'columns', inplace=True)

In [ ]:
#3 calcolo intervalli di tempo

time_unit= 60*60*24 #1 giorno


#considero solo gli intervalli di tempo positivi
mask = (join1['endTime'] > join1['prevTime'])
join1['intervals'] = 0 #inizializzo a 0
join_valid = join1[mask]
join1.loc[mask,'intervals'] = join_valid['endTime'] - join_valid['prevTime'] #calcolo intervalli positivi
join1.loc[mask,'intervals'] = join1['intervals'] // time_unit #cambio unità di misurA da s a time_unit

join_valid = join1[mask]

In [ ]:
#4 creo istogramma frequenze

plt.gcf().set_size_inches(25,10) #imposto dimesioni figura

bin_size=20 #numero di giorni (relativi alla durata dell' intervallo) per bin

plt.title("Distribuzione degli intervalli di tempo che intercorrono tra la transazione che genera un valore in output (UTXO) e quella che lo consuma",fontsize=18)
plt.xlabel("Durata di tempo intervallo (giorni)",fontsize=14)
plt.ylabel("Numero di intervalli",fontsize=14)

plt.hist(join_valid['intervals'],bins=join_valid['intervals'].max()//bin_size,log=True,edgecolor='black')

plt.show()

![](ImmaginiLWS/2.1d.png)

## e) Analisi a piacere: Numero di transazioni e quantità di fee medie, per ognuno dei 12 mesi, sull' intero periodo

### Cosa fare

1. Ottenere mese di ogni transazione
2. Raggruppare per ogni mese e calcolare numero transazioni e somma fee
3. Plottare su bar plot

#### Perché può essere interessante

* Permette di vedere se un maggior numero di transazioni implica sempre un maggior numero di fee. 
* Permette di vedere se in determinati mesi (o una determinata stagione) si registra un maggior numero di transazioni/ maggior quantità di fee. Se, per esempio, ci fosse un picco nei mesi di Giugno/Luglio/Agosto potremmo osservare che il mercato dei Bitcoin è più attivo d'estate.

In [ ]:
#1
df_transactions['month'] = pd.to_datetime(df_transactions['timestamp'],unit='s').dt.month 

#2

months_table= df_transactions.groupby('month').agg(
    totFees= ('fee', np.sum),
    totTx= ('txId','size')
).reset_index() 

In [ ]:
#3

#imposto assi

x=months_table['month']
months_list= [calendar.month_name[el] for el in x] #etichette per asse x
y1= (months_table['totFees']/4).apply(np.log)
y2= (months_table['totTx']/4).apply(np.log) #2009-12 sono 4 anni quindi /4
width=0.2 #larghezza barre

plt.gcf().set_size_inches(25,14) #imposto dimesioni figura

plt.bar(x-0.1,y1,width,color='blue',label='fees')
plt.bar(x+0.1,y2,width,color='orange',label='Tx')
plt.xticks(x,months_list,fontsize=18)
plt.yticks(np.arange(0,26,1))

plt.title("Numero di transazioni e quantità di fee per ogni mese",fontsize=24)
plt.xlabel('months',fontsize=18)
plt.ylabel('quantity (ln)',fontsize=18)
plt.legend(fontsize=14)
plt.show()

![](ImmaginiLWS/2.1e.png)

# Esercizio 2.2

In [ ]:
import pandas as pd
import networkx as nx
from collections import defaultdict
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df_inputs = pd.read_csv("dataset/inputs.csv")
df_outputs = pd.read_csv("dataset/outputs.csv")
df_mapping = pd.read_csv("dataset/mapAddr2Ids8708820.csv")

## Implementare un algoritmo di clustering che realizzi l'euristica descritta. 

* Viste le dimensioni del DataSet è necessario limitare la complessità dell’algoritmo, in particolare si dovrà implementare un algoritmo di complessità lineare rispetto al numero di indirizzi diversi contenuti nel DataSet.

### Spiegazione dell'algoritmo

Dal punto di vista teorico, l'algoritmo di clustering per l' euristica richiesta si basa sul creare un grafo a partire dai dati, e si compone delle seguenti fasi:

* Aggiungere tutti gli indirizzi come nodi nel grafo;
* Per ogni transazione, aggiungere un arco dal primo input verso tutti gli altri input della transazione;
* Calcola le componenti debolmente connesse.

Le componenti debolmente connesse ottenute da quest'algoritmo saranno proprio i cluster secondo l'euristica richiesta. Per implementare quest' algoritmo però bisogna effettuare alcuni passaggi in più di manipolazione dati su tabelle. I passaggi che avvengono nell'implementazione sono i seguenti:

1. Effettuare join fra tabelle outputs e mapping, per recuperare hash di ogni output;
2. Effettuare join fra tabella ottenuta da *1* e tabella inputs( è l'oggetto df_inputs, riuso le stesse variabili per risparmiare in memoria) ottenendo le coppie (txId,hash), che mi indicano che quell' hash è input di quella transazione;
3. Per ogni riga in df_inputs:
    1. se è la prima volta che vedo la transazione, inserisco in una mappa la coppia (txId,hash) di quella riga;
    2. se la transazione è presente nella mappa (vista almeno una volta), recupero l'hash relativo e aggiungo l'arco (primo hash, hash corrente) al grafo;
4. Aggiungo al grafo i rimanenti nodi isolati scorrendo la tabella mapping;
5. Cerco le componenti connesse (il grafo che ho creato è indiretto; tuttavia cercare componenti connesse su grafi indiretti è la stessa cosa che cercare componenti debolmente connesse su grafi diretti);

#### Analisi

Ecludendo le join (merge) iniziali l'algoritmo scorre tutta la tabella di input nello step *3*. Per ogni entry della tabella, controlla se il txId (chiave) era già prensente del dictionary. Questa operazione, è $O(1)$, visto che i dizionari sono implementati con una hashtable. Pertanto il passo $3$ ha complessità $O(Inputs)$. (Assumendo il caso medio per la complessità di tempo delle hashtable)

Il passo 4 ha complessità $O(Address)$, visto che scorro tutta la tabella degli indirizzi.

Il passo 5, non sapendo come è implementato su NetworkX, ha, alla peggio, complessità $O(Address + Inputs)$. (Informandomi però probabilmente ha solo complessità $O(Address)$)

Pertanto l'algoritmo ha complessità $O(Inputs + Address)$.

In [ ]:
start=time.process_time()
print("started")


#1
df_outputs= df_outputs.merge(df_mapping,on='addressId') #join passo 1
df_outputs= df_outputs[['txId','position','hash']] #proietto solo colonne interessanti

#2
df_inputs= df_inputs.merge(df_outputs, left_on=['prevTxId','prevTxpos'], right_on= ['txId','position']) #join passo 2
df_inputs= df_inputs[['txId_x','hash']] #proietto solo colonne interessanti
df_inputs= df_inputs.rename({'txId_x' : 'txId'},axis='columns') #rinomino per chiarezza

#txId mi dice di che transazione sono input
#hash è l'hash associato

#Dictionary che contiene coppie (txId,primo hash relativo)
tx_map = defaultdict(lambda : "")

G= nx.Graph()

print("Start loop")


#3
for row in df_inputs.itertuples(name=None):
    
    curr= tx_map[row[1]] #accedo a tx_map['txId'], mi restituisce hash relativo
    
    if curr == "" : #priva volta vedo questa transazione, non devo aggiungere archi
        tx_map[row[1]]= row[2] #row[2] contiene hash
        continue
        
    #avevo già visto un hash relativo a questa transazione    
    #curr sarà quell'hash
    #devo avere arco in G frai 2 hash
    
    G.add_edge(curr,row[2])
    
#a questo punto ho aggiunto tutte le componenti connesse, devo aggiungere i nodi isolati
#step 4
G.add_nodes_from(df_inputs['hash'])

#Grafo completato, ora devo trovare tutte le componenti debolmente connesse 

print("Start components")

#step 5

my_components= [c for c in sorted(nx.connected_components(G),key=len,reverse=True)]
my_components_len = [len(c) for c in my_components]

end=time.process_time()
print(end-start) #187.2203370000002


#per fare dump grafo scommentare le due righe sotto; ci mette tanto

#with open("Graph.json",'w') as outfile :
 #   json.dump(nx.adjacency_data(G),outfile)

In [ ]:
#per fare dump cluster

with open('Components.json','w') as outfile:
    json.dump([list(c) for c in my_components],outfile)
    

## Produrre alcune statistiche descrittive del clustering ottenuto (dimensione media, minima e massima dei cluster, distribuzione delle loro dimensioni, etc.)

In [ ]:
#numero di cluster
print(f"number of clusters: {len(my_components_len)}")

#dimensione massima
print(f"max cluster size: {my_components_len[0]}")

#dimensione minima
print(f"min cluster size: {my_components_len[-1]}")

mean= sum(my_components_len)/len(my_components)

print(f"average cluster size: {mean}")

#percentuale cluster di dimensione 1
ones= len([1 for c_len in my_components_len if c_len == 1])
print(f"Percentage of clusters with 1 node only: {ones/len(my_components_len)*100 : .2f}")

#distribuzione dimensioni

#calcolo frequenze

appearances = defaultdict(int)

for curr in my_components_len:
    appearances[curr] += 1

#plot
    
plt.gcf().set_size_inches(25,14) #imposto dimesioni figura
plt.xscale('log')
plt.ylabel('numero di cluster',fontsize=18)
plt.xlabel('Dimensione dei cluster',fontsize=18)
plt.title('Distribuzione della dimensione dei cluster',fontsize=24)

plt.xscale('log') #scala log
plt.yscale('log')

plt.plot([key for key in appearances],list(appearances.values()),marker='x',color='black',lw=0)

plt.show()

![](ImmaginiLWS/2.2.png)

# Esercizio 2.3

## WalletExplorer

### Cosa fare

Le pagine web di WalletExplorer non fanno uso di javascript, pertanto beautifulsoup risulta sufficiente.

Per capire cosa fare, iniziamo guardando la pagina iniziale di WalletExplorer. Notiamo che sono presenti due barre di ricerca, una nel centro della pagina e una nella toolbar in alto a dx.


![](ImmaginiLWS/WE1.png)

Provando a interagire per ricercare indirizzi possono accadere due cose:

* l' indirizzo è stato trovato con successo, e verrà mostrata la pagina del wallet relativo
* l'indirizzo non è presente, in quel caso verrà mostrata la seguente pagina:

![](ImmaginiLWS/WE2.png)

Notiamo nella barra di ricerca browser la query che è stata effettuata al server: non è altro che l'assegnamento al parametro 'q' dell'address ricercato. Pertanto nel nostro script potremo riassegnare ogni voltaquel parametro, senza dover interagire con i form della pagina principale.

Quando invece una ricerca di un address va a buon fine, verremo portati alla pagina del wallet di cui fa parte.

![](ImmaginiLWS/WE3.png)

Analizzando il codice HTML relativo, notiamo che il nome del wallet si trova all'interno di un header h2 (in particolare è il terzo figlio). Pertanto a noi basta recuperare l' header h2, e il terzo figlio sarà proprio il nome del wallet che stavamo cercando per la procedura di deanonimizzazione

![](ImmaginiLWS/WE4.png)

Putroppo non sempre, dopo aver eseguito questa procedura, WalletExplorer restituirà un nome significativo. In alcuni casi è possibile restituisca una sequenza del tipo '[numero_esadecimale]'.

![](ImmaginiLWS/WE5.png)

Data l'impossibilità di fare un' analisi su tutti gli indirizzi di ogni cluster ( dopo poco tempo che si prova a fare richieste WalletExplorer blocca l'indirizzo ip) lo script, come riprova, recupera il numero di address associato al cluster su WalletExplorer. 

Per fare ciò notiamo che, nella pagina di un wallet è presente, accanto al nome del wallet un link che rimanda alla pagina dove sono presenti tutti gli address relativi.

![](ImmaginiLWS/WE6.png)

**Attenzione**: non tutte le pagine presentano solo il link agli address, alcune presentano anche un link al servizio/organizzazione relativo). In questo caso il link agli address del wallet sarà l'ultimo dei link presenti nell' header h2

![](ImmaginiLWS/WE9.png)

Se proviamo ad andare su questo link, verremo rimandati alla pagina degli address di quel wallet,
dove è presente la scritta '(total addresses: numero_address)'

![](ImmaginiLWS/WE7.png)

Analizzando il codice HTML della pagina, notiamo che il numero di address si trova dentro l'unico tag \<small> all'interno del primo div con $class = paging$ all'interno del div con $id = main$, potendolo così recuperare facilmente.

![](ImmaginiLWS/WE8.png)

In [ ]:
import json
import requests
from bs4 import BeautifulSoup

In [ ]:
#importo cluster che mi ero creato nel punto 2.2

with open('Components.json','r') as infile:
    my_components= json.load(infile)
my_components_len= [len(c) for c in my_components]

In [ ]:
#prendo i 10 cluster più grandi
n= 10 #numero cluster

chosen_clusters = my_components[:n]

In [ ]:
#funzione che mi restituisce il nome del wallet su 'https://www.walletexplorer.com/' relativo al cluster 
def solve_name(cluster):
    
    base_url = 'https://www.walletexplorer.com/' #url base del sito
    
    #headers per fingersi umano,
    #non necessari ma li ho usati per cercare di aggirare il rate massimo di richieste (senza successo)
    headers= {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:101.0) Gecko/20100101 Firefox/101.0'}
    
    for address in cluster: #per ogni indirizzo del cluster
        
        try : 
            
            #effettuo query per trovare wallet address
            r= requests.get(base_url,params={'q' : address},headers=headers) 
            
            #controllo se richiesta avvenuta con successo, solleva eccezione trattata in except
            r.raise_for_status() 
            
            
            bs= BeautifulSoup(r.text,'html.parser')
            error= bs.find('p',class_='error') #controllo se ha trovato wallet address
            #la find mi dice se è presente il paragrafo rosso, che indica fallimento, 
            
            if error is not None: #impossibile trovare wallet
                continue #vado a quello dopo
            
            #wallet trovato
            #recupero header h2
            title= bs.find('h2')
            
            
            #parte codice commentata per cercare di deanonimizzare, saltando nomi non significativi (senza successo)
            #if list(title.children)[2][0] == '[':
            #    continue
            
            #nome del wallet è il terzo figlio
            wallet_name= list(title.children)[2]
            
            #cerco tutti i link nell'header h2
            a=title.find_all('a',href=True) 
            
            
            #l'ultimo link è quello della pagina degli address, procedo a richiederla
            r=requests.get(base_url+a[-1]['href'],headers=headers)
            bs= BeautifulSoup(r.text,'html.parser')
            
            add_num= bs.find('div',id='main').find('div',class_='paging').find('small').text #numero address wallet
            
            return (wallet_name,add_num) #restituisco nome e dimensioni
            
        except requests.exceptions.RequestException as e: #errore di qualche tipo
            print(e)
            continue
    
    return None        
    


In [ ]:
#deanonimizzo i cluster scelti

names_list_we= list()

for idx,cluster in enumerate(chosen_clusters):
    print(idx)
    
    names_list_we.append(solve_name(cluster))
    
    print(names_list_we[idx])



In [ ]:
#mostro nomi e dimensioni dei cluster

for idx,(my_len,(name,new_len))in enumerate(zip(my_components_len,names_list_we)):
    print(f'Cluster {idx} size: {my_len} \tName: {name}\t\t{new_len}')
    

![](ImmaginiLWS/res1.png)

## Bitcoininfocharts

Sebbene Bitinfocharts presenti diverse parti in javascript, il nome del wallet relativo ad un indirizzo è già presente nel codice html della pagina, pertanto beautifulsoup risulta sufficiente. Tuttavia Bitinfocharts fa uso di captcha, che vengono creati dinamicamente. 

Sono proposti due script. Il **primo**,realizzato con beautifulsoup, mostra che la deanonimizzazione è possibile senza selenium ( lo script rimarrà bloccato indefinitamente in caso di captcha). Il **secondo**, realizzato con selenium, quando incontra un captcha passa all'indirizzo successivo, nel tentativo di non essere più bloccato.

Prima di mostrare il codice però osserviamo cosa il nostro script dovrà fare. Su Bitinfocharts gli indirizzi hanno una propria pagina, nella quale è presente un riferimento al wallet di cui fanno parte.

![](ImmaginiLWS/BI1.png)

Osservando il sorgente della pagina, notiamo che il link al wallet è il primo link dentro la prima tabella di classe "*table table-striped table-condensed*" presente nella pagina. Notare che è già presente il nome del wallet, pertanto non è necessario dover andare sulla pagina del wallet per reperirlo (evitando il più possibile captcha).

![](ImmaginiLWS/BI2.png)

In [ ]:
import json
import requests
from bs4 import BeautifulSoup

from collections import Counter

from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.common.exceptions import NoSuchElementException

In [ ]:
with open('Components.json','r') as infile:
    my_components= json.load(infile)
my_components_len= [len(c) for c in my_components]

In [ ]:
#prendo i 10 cluster più grandi
n= 10 #numero cluster

chosen_clusters = my_components[:n]

**Note sullo script:**

* BitInfocharts blocca qualunque automaticamente qualunque script/bot osservando lo user-agent. Si rivela quindi necessario cambiarlo, in modo da potersi fingere utenti umani.

* Similmente a WalletExplorer, Bitinfocharts ha dei wallet con un nome non significato, e quindi non l'ideale per la deanonimizzazione (in questo caso è un numero intero). Data la difficoltà nel visionare un gran numero di pagine (dovuta ai captcha) lo script permette di impostare un limite di indirizzi da analizzare (parametro *limit*).

* Nel caso in cui non si sia trovato nessun nome significativo, viene preso come wallet relativo al cluster quello di cui fanno parte più indirizzi analizzati.

* Nel caso di captcha lo script rimarrà bloccato sulla chiamata **requests.get**

In [ ]:
def solve_name_bs(cluster,limit=80):
    
    #user agent da usare per non essere bloccati
    headers= {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:101.0) Gecko/20100101 Firefox/101.0'}
    base_url='https://bitinfocharts.com/bitcoin/address/'
    
    #lista wallet non significativi visti
    results_seen=list()
    
    
    for address in cluster:
        
        if limit == 0 : #se ho raggiunto limit iteraioni
            
            
            if len(results_seen) == 0: #caso estremo, accade se nessun indirizzo passato è presente nel sito
                return None
            data = Counter(results_seen)
            return data.most_common(1)[0][0]
        
        limit = limit-1
        
        #richiedo pagina address
        r= requests.get(base_url+address,headers=headers)
        
        try:
            r.raise_for_status() #controllo che mi sia stata data effettivamente la pagina
        except requests.exceptions.HTTPError as e:
            continue

        bs= BeautifulSoup(r.text,'html.parser')
        table= bs.find('table',class_="table table-striped table-condensed") #prendo tabella interessata
        a=table.find('a') #prendo link al wallet
        
        try:
            #se il wallet è un numero intero -> non significativo, continua a cercare
            int(a.text[a.text.find('wallet: ')+8:])
            results_seen.append(a.text[a.text.find('wallet: ')+8:]) #metti wallet in lista
            continue
        
        except ValueError: #non è un numero -> nome significativo
            return a.text[a.text.find('wallet: ')+8:]
        

In [ ]:
names_list=list()


for idx,cluster in enumerate(chosen_clusters):
    print(idx)
    names_list.append(solve_name_bs(cluster))

**Note sullo script:**

* il funzionamento è analogo allo script con bs, ma, nell' eventualità di trovare un captcha, lo script con selenium salta al prossimo indirizzo nella speranza di non essere bloccato di nuovo.

In [ ]:
def solve_name_sel(cluster,limit=80):
    base_url='https://bitinfocharts.com/bitcoin/address/'

    #commentare  queste 2 righe pervedere finestra browser
    options = FirefoxOptions()
    options.add_argument("--headless")
    
    #uso firefox perchè più veloce sul mio pc
    driver = webdriver.Firefox(options=options)

    #lista wallet non significativi visti
    results_seen=list()

    
    for address in cluster:
        
        if limit == 0 :  # iterato limit volte
            driver.quit() #chiudi webdriver
            
            #non sono riuscito a ottenere nessun wallet (causa captcha, pagine inesistenti)
            if len(results_seen) == 0: 
                return None
            
            #ho visto almeno un wallet
            #restituisco wallet più frequente
            data = Counter(results_seen)
            return data.most_common(1)[0][0]
        
        
        limit= limit-1

        driver.get(base_url+address) #richiedi pagina
        
        #print(driver.title)
        
        #se ottengo captcha vado avanti
        if driver.title =="Just a moment..." or driver.title == "Ci siamo quasi…":
            continue
        
        #ottengo link wallet
        a= driver.find_element(By.XPATH,'/html/body/div[3]/div[3]/table/tbody/tr/td/table/tbody/tr[1]/td[2]/small/a') 
        
        #prendo il nome nel link
        link= a.get_attribute('href') 
        
        try :
            
            #se il nome è un intero -> non significativo -> mettilo in lista e vai avanti
            int(link[link.find('wallet/')+len('wallet/'):])
            results_seen.append(link[link.find('wallet/')+len('wallet/'):])
            continue
            
        except ValueError: #è un nome significativo
            driver.quit()    
            return link[link.find('wallet/')+len('wallet/'):]
        
        
        #driver.quit()    
        #return link[link.find('wallet/')+len('wallet/'):]
        
    return None

In [ ]:
#per vedere risultati script beautifulsoup non eseguire questa cella. Comunque drovebbero essere uguali

names_list=list()


for idx,cluster in enumerate(chosen_clusters):
    print(idx)
    names_list.append(solve_name_sel(cluster))

In [ ]:
for idx,(my_len,name) in enumerate(zip(my_components_len,names_list)):
    print(f'Cluster {idx} size: {my_len} \tName: {name}')

![](ImmaginiLWS/res2.png)

## Perchè i nomi sono diversi diversi?

* WalletExplorer de-anonimizza gli indirizzi utilizzando una procedura di clustering e scraping. Una possibile ragione per cui WalletExplorer potrebbe commettere errori nella de-anonimizzazione è l'euristica utilizzata. Nonostante l'euristica sia scelta per minimizzare gli errori di de-anonimizzazione, potrebbe verificarsi una discrepanza nei nomi rispetto a quelli forniti da Bitinfocharts.

* Non è chiaro come Bitinfocharts ottenga i nomi dei wallet. Pertanto, se la fonte o il processo da cui ottengono i nomi è errato, potrebbero esserci differenze nei nomi rispetto a quelli effettivamente associati a quel cluster. Questo significa che i nomi ottenuti tramite WalletExplorer potrebbero essere diversi da quelli ottenuti con Bitinfocharts.